# Solving the "Fantasy Football Knapsack Problem"

The [Knapsack Problem](https://en.wikipedia.org/wiki/Knapsack_problem) is a combinatorial optimization problem. In the "fantasy football" variation of the knapsack problem, we are trying to determine an optimal selection of players to maximize our team's expected fantasy points. There are two major components to playing fantasy football "quantitatively." The first is building statistical models to predict how many fantasy points a player will produce. The second is determining which players to pick, given their predicted point production and their cost. This notebook will show two approaches for tackling the second task.

In [3]:
import csv

In [8]:
class Player():
    def __init__(self, position, name, salary, points, value):
        self.self = self
        self.position = position
        self.name = name
        self.salary = salary
        self.points = points
        self.value = value
        
    def __iter__(self):
        return iter(self.list)
    
    def __str__(self):
        return "{} {} {} {}".format(self.name,self.position,self.salary, self.points)

In [9]:
# This csv contains our predictions and salaries for each player. 
# We parse each row of the csv and convert it into a Player object.
with open('DKSalaries.csv', 'r') as data:
    reader = csv.reader(data)
    reader.next()
    players = []
    for row in reader:
        name = row[1]
        position = row[0]
        salary = int(row[2])
        points = float(row[4])
        value = points / salary  
        player = Player(position, name, salary, points, value)
        players.append(player)

## The Greedy Approach

Greedy Algorithms are quite simple. You prioritize a list of objects, and then you select the objects in that order, as long as they don't violate some constraint. For our scenario, the constraints are the overall budget you can spend on your team, and the required number of players in each position. The tricky part in developing a greedy algorithm is determining the correct way to prioritize the objects. 

One way that seems reasonable to prioritize players is by their expected point production. We would select the available player that we predict to have the best game, as long as we can afford them, and have room in their position group. Let's see what type of team this approach creates:

In [10]:
def points_knapsack(players):
    budget = 50000
    current_team_salary = 0
    constraints = {
        'QB':1,
        'RB':2,
        'WR':3,
        'TE':1,
        'DST':1,
        'FLEX':1
        }
    
    counts = {
        'QB':0,
        'RB':0,
        'WR':0,
        'TE':0,
        'DST':0,
        'FLEX':0
        }
    
    players.sort(key=lambda x: x.points, reverse=True)
    team = []
    
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if counts[pos] < constraints[pos] and current_team_salary + sal <= budget:
            team.append(player)
            counts[pos] = counts[pos] + 1
            current_team_salary += sal
            continue
        if counts['FLEX'] < constraints['FLEX'] and current_team_salary + sal <= budget and pos in ['RB','WR','TE']:
            team.append(player)
            counts['FLEX'] = counts['FLEX'] + 1
            current_team_salary += sal 

    return team

In [11]:
team = points_knapsack(players)
points = 0
salary = 0
for player in team:
    points += player.points
    salary += player.salary
    print player
print "\nPoints: {}".format(points)
print "Salary: {}".format(salary)

Julio Jones WR 9200 29.7
Le'Veon Bell RB 8500 28.6
Tom Brady QB 7800 28.527
Larry Fitzgerald WR 6800 27.05
Devonta Freeman RB 6300 26.7
Antonio Brown WR 8700 26.2
Lions  DST 2500 10.75

Points: 177.527
Salary: 49800


### Oops.

There's a slight problem here: **we didn't end up with enough players on our team.** A valid team has 9 players, ours only has 7. After picking 7 players, we have used \$49,800 of our \$50,000 budget. We don't have enough money left to afford even the cheapest available player, and we still need *two* more players. There are ways around this: rather than adding a player as long as he doesn't put us over the budget, we could make sure that after adding him we still had the budget left to fill out our team with the cheapest possible players at each position.

This approach would leave us with a very "top-heavy" team: a few really good players, and a few "bottom of the barrel" players. This isn't necessarily a bad thing, but let's try a different approach.

### Prioritizing by Points per Dollar

Rather than prioritizing by expected point production, let's try prioritizing by **expected points per dollar of cost.**

In [12]:
def value_knapsack(players):
    budget = 50000
    current_team_salary = 0
    constraints = {
        'QB':1,
        'RB':2,
        'WR':3,
        'TE':1,
        'DST':1,
        'FLEX':1
        }
    
    counts = {
        'QB':0,
        'RB':0,
        'WR':0,
        'TE':0,
        'DST':0,
        'FLEX':0
        }
    
    players.sort(key=lambda x: x.value, reverse=True)
    team = []
    
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if counts[pos] < constraints[pos] and current_team_salary + sal <= budget:
            team.append(player)
            counts[pos] = counts[pos] + 1
            current_team_salary += sal
            continue
        if counts['FLEX'] < constraints['FLEX'] and current_team_salary + sal <= budget and pos in ['RB','WR','TE']:
            team.append(player)
            counts['FLEX'] = counts['FLEX'] + 1
            current_team_salary += sal 

    return team

In [13]:
team = value_knapsack(players)
points = 0
salary = 0
for player in team:
    points += player.points
    salary += player.salary
    print player
print "\nPoints: {}".format(points)
print "Salary: {}".format(salary)

Austin Seferian-Jenkins TE 3600 17.95
Ladarius Green TE 3200 14.467
Travis Benjamin WR 4500 20.2
Broncos  DST 3600 16.0
Andy Dalton QB 5700 24.295
Devonta Freeman RB 6300 26.7
Keshawn Martin WR 3000 12.3
Dion Lewis RB 4800 19.5
Larry Fitzgerald WR 6800 27.05

Points: 178.462
Salary: 41500


### Hmm.

We got a valid team, but it doesn't seem quite "optimal." We ended up using only \$41,500 of our \$50,000 budget. We optimized for "value" but that's not exactly what we want, we want to optimize for **points.**

### Improving on Points per Dollar

This approach seems promising though; we have a team full of under-valued players. We can try to replace some of the under-valued players that aren't predicted to get many points with some players with worse points per dollar ratios, but more expected points.

In [14]:
def improved_knapsack(players):
    budget = 50000
    current_team_salary = 0
    constraints = {
        'QB':1,
        'RB':2,
        'WR':3,
        'TE':1,
        'DST':1,
        'FLEX':1
        }
    
    counts = {
        'QB':0,
        'RB':0,
        'WR':0,
        'TE':0,
        'DST':0,
        'FLEX':0
        }
    
    players.sort(key=lambda x: x.value, reverse=True)
    team = []
    
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if counts[pos] < constraints[pos] and current_team_salary + sal <= budget:
            team.append(player)
            counts[pos] = counts[pos] + 1
            current_team_salary += sal
            continue
        if counts['FLEX'] < constraints['FLEX'] and current_team_salary + sal <= budget and pos in ['RB','WR','TE']:
            team.append(player)
            counts['FLEX'] = counts['FLEX'] + 1
            current_team_salary += sal 
    
    players.sort(key=lambda x: x.points, reverse=True)
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if player not in team:
            pos_players = [ x for x in team if x.position == pos]
            pos_players.sort(key=lambda x: x.points)
            for pos_player in pos_players:
                if (current_team_salary + sal - pos_player.salary) <= budget and pts > pos_player.points:
                    team[team.index(pos_player)] = player
                    current_team_salary = current_team_salary + sal - pos_player.salary
                    break
    return team

In [15]:
team = improved_knapsack(players)
points = 0
salary = 0
for player in team:
    points += player.points
    salary += player.salary
    print player
print "\nPoints: {}".format(points)
print "Salary: {}".format(salary)

Austin Seferian-Jenkins TE 3600 17.95
Ladarius Green TE 3200 14.467
Travis Benjamin WR 4500 20.2
Broncos  DST 3600 16.0
Andy Dalton QB 5700 24.295
Le'Veon Bell RB 8500 28.6
Julio Jones WR 9200 29.7
Dion Lewis RB 4800 19.5
Larry Fitzgerald WR 6800 27.05

Points: 197.762
Salary: 49900


### Looks much better.

We were able to swap some lower-performing players out with more expensive and higher-producing ones, while still staying under the budget. Out goes Mark Sanchez, in comes Tom Brady. **We improved our total expected point output by about 20 points.**

### But wait...

Can't we just enumerate all possible teams, and find the optimal team that way? Technically yes, realistically no. 

Simplifying the problem a bit, we can approximate the number of possible teams as '450 choose 9'
That comes out to **1923912000000000000000000 possibilities**

The Knapsack optimization problem is [NP-Hard](https://en.wikipedia.org/wiki/NP-completeness), meaning "there is no known polynomial algorithm which can tell, given a solution, whether it is optimal" (unless [P=NP](https://en.wikipedia.org/wiki/P_versus_NP_problem)).

The greedy approach we've taken is an approximation algorithm. We are not claiming to find the perfectly optimal team, just our best approximation. Let's take a look at another method.

## Genetic Algorithm

[Genetic algorithms](https://en.wikipedia.org/wiki/Genetic_algorithm) are often effective for optimization problems. Genetic algorithms mimic natural selection, and "generate solutions to optimization problems using techniques inspired by natural evolution, such as inheritance, mutation, selection, and crossover."

There exist several Python libraries that help with implementing genetic algorithms ([DEAP](http://deap.readthedocs.org/en/master/), [PyEvolve](http://pyevolve.sourceforge.net/)) but we're going to implement one ourselves.

In [16]:
import random
import math
from operator import add
import matplotlib.pyplot as plt
%matplotlib inline

Genetic algorithms start by generating a population of random individuals. In this context, an individual is actually a team, and a population is a collection of many teams.

In [17]:
def CreateRandomTeam():    
    team = {
    'qb' : random.sample(qbs,1),
    'rb' : random.sample(rbs,2),
    'wr' : random.sample(wrs,3),
    'te' : random.sample(tes,1),
    'flex' : random.sample(flexs,1),
    'dst' : random.sample(dsts,1)
    }
    
    while True:
        flexer = team['flex'][0]
        if flexer in team['rb'] or flexer in team['wr'] or flexer in team['te']:
            team['flex'] = random.sample(flexs,1)
        else:
            break
    
    return team

In [18]:
def GetTeamPointTotal(team):
    total_points = 0
    for pos, players in team.iteritems():
        for player in players:
            total_points += player.points
    return total_points

def GetTeamSalary(team):
    total_salary = 0
    for pos, players in team.iteritems():
        for player in players:
            total_salary += player.salary
    return total_salary

def printTeam(team):
    print team['qb'][0]
    print team['rb'][0]
    print team['rb'][1]
    print team['wr'][0]
    print team['wr'][1]
    print team['wr'][2]
    print team['te'][0]
    print team['flex'][0]
    print team['dst'][0]

In [19]:
def CreatePopulation(count):
    return [CreateRandomTeam() for i in range(0,count)]

In [20]:
qbs = [player for player in players if player.position == 'QB']
rbs = [player for player in players if player.position == 'RB']
wrs = [player for player in players if player.position == 'WR']
tes = [player for player in players if player.position == 'TE']
dsts = [player for player in players if player.position == 'DST']
flexs = [player for player in players if player.position != 'DST' and player.position != 'QB']

### Fitness
In the greedy approach, the only part that really required some creativity from us was determining how to prioritize the players. In the genetic approach, we must come up with our [Fitness function](https://en.wikipedia.org/wiki/Fitness_function). Genetic algorithms follow the "only the strong survive" philosophy. Our fitness function tells the algorithm which teams are strong. Our fitness function is very simple: return the number of points you expect a team to produce (the higher the better), *unless* that team is over budget, then return zero.

In [21]:
def fitness(team):
    points = GetTeamPointTotal(team)
    salary = GetTeamSalary(team)
    values = team.values()
    if salary > 50000:
        return 0
    return points

In [22]:
def grade(pop):
    'Find average fitness for a population.'
    summed = reduce(add, (fitness(team) for team in pop))
    return summed / (len(pop) * 1.0)

In [23]:
def listToTeam(players):
    return {
    'qb' : [players[0]],
    'rb' : players[1:3],
    'wr' : players[3:6],
    'te' : [players[6]],
    'flex' : [players[7]],
    'dst' : [players[8]]
}

### Breeding
Another important aspect of genetic algorithms is breeding. Once we can determine which teams are strongest, we "breed" them together to create new teams. This process is also called Cross-Over. 

In [24]:
def breed(mother, father):
    positions = ['qb','rb','wr','te','flex','dst']
    
    mother_lists = [mother['qb'] + mother['rb'] + mother['wr'] + mother['te'] + mother['flex'] + mother['dst']]
    mother_list = [item for sublist in mother_lists for item in sublist]
    father_lists = [father['qb'] + father['rb'] + father['wr'] + father['te'] + father['flex'] + father['dst']]
    father_list = [item for sublist in father_lists for item in sublist]

    index = random.choice([1,3,6,7,8])
    child1 = listToTeam(mother_list[0:index] + father_list[index:])
    child2 = listToTeam(father_list[0:index] + mother_list[index:])
    
    while True:
        flexer = child1['flex'][0]
        if flexer in child1['rb'] or flexer in child1['wr'] or flexer in child1['te']:
            child1['flex'] = random.sample(flexs,1)
        else:
            break
            
    while True:
        flexer = child2['flex'][0]
        if flexer in child2['rb'] or flexer in child2['wr'] or flexer in child2['te']:
            child2['flex'] = random.sample(flexs,1)
        else:
            break
        
    return[child1, child2]  

### Mutation
Mutation is used to randomly shake up some teams. This prevents us from getting stuck at a local maximum.

In [25]:
def mutate(team):
    positions = ['qb','rb','wr','te','flex','dst']
      
    random_pos = random.choice(positions)
    if random_pos == 'qb':
        team['qb'][0] = random.choice(qbs)
    if random_pos == 'rb':
        team['rb'] = random.sample(rbs,2)
    if random_pos == 'wr':
        team['wr'] = random.sample(wrs,3)
    if random_pos == 'te':
        team['te'][0] = random.choice(tes)
    if random_pos == 'flex':
        team['flex'][0] = random.choice(flexs)
    if random_pos == 'dst':
        team['dst'][0] = random.choice(dsts)
        
    while True:
        flexer = team['flex'][0]
        if flexer in team['rb'] or flexer in team['wr'] or flexer in team['te']:
            team['flex'] = random.sample(flexs,1)
        else:
            break
    return team

### Evolution
Evolution is where the action happens in a genetic algorithm. Initially we create a population of entirely random teams. Then, each generation, the strongest teams are bred together, the weakest are discarded, some more random teams are created, and some teams are mutated. 

In [1]:
def evolve(pop, retain=0.35, random_select=0.05, mutate_chance=0.005):
    graded = [ (fitness(team), team) for team in pop]
    graded = [ x[1] for x in sorted(graded, reverse=True)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]

    # randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random.random():
            parents.append(individual)

    # mutate some individuals
    for individual in parents:
        if mutate_chance > random.random():
            individual = mutate(individual)

    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = random.randint(0, parents_length-1)
        female = random.randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            babies = breed(male,female)
            for baby in babies:
                children.append(baby)
    parents.extend(children)
    return parents

In [31]:
best_teams=[]
history = []
p = CreatePopulation(10000)
fitness_history = [grade(p)]
for i in xrange(40):
    p = evolve(p)
    fitness_history.append(grade(p))
    valid_teams = [ team for team in p if GetTeamSalary(team) <= 50000]
    valid_teams = sorted(valid_teams, key=GetTeamPointTotal, reverse=True)
    if len(valid_teams) > 0:
        best_teams.append(valid_teams[0])
for datum in fitness_history:
    history.append(datum)

best_teams = sorted(best_teams, key=GetTeamSalary, reverse=True)
choice = best_teams[0]
printTeam(choice)
print GetTeamSalary(choice)
print GetTeamPointTotal(choice)

Andy Dalton QB 5700 24.295
Le'Veon Bell RB 8500 28.6
Devonta Freeman RB 6300 26.7
Tavon Austin WR 4300 14.875
Travis Benjamin WR 4500 20.2
Jeremy Maclin WR 6000 19.9
Austin Seferian-Jenkins TE 3600 17.95
Rob Gronkowski TE 7500 25.6
Broncos  DST 3600 16.0
50000
194.12


### Let's take a look.
The team generated by our genetic algorithm is expected to produce almost exactly the same number of points as our greedy approach, though slightly less. One key distinction between these two approaches is that the greedy approach will generate the same team everytime you run it, whereas the genetic approach, because of all the randomness involved, will not. To prove it, I've copied the above cell to the cell below, and let's run it again...

In [32]:
best_teams=[]
history = []
p = CreatePopulation(10000)
fitness_history = [grade(p)]
for i in xrange(40):
    p = evolve(p)
    fitness_history.append(grade(p))
    valid_teams = [ team for team in p if GetTeamSalary(team) <= 50000]
    valid_teams = sorted(valid_teams, key=GetTeamPointTotal, reverse=True)
    if len(valid_teams) > 0:
        best_teams.append(valid_teams[0])
for datum in fitness_history:
    history.append(datum)

best_teams = sorted(best_teams, key=GetTeamSalary, reverse=True)
choice = best_teams[0]
printTeam(choice)
print GetTeamSalary(choice)
print GetTeamPointTotal(choice)

Tyrod Taylor QB 5800 21.155
Devonta Freeman RB 6300 26.7
Le'Veon Bell RB 8500 28.6
Travis Benjamin WR 4500 20.2
Keenan Allen WR 7200 23.675
Marlon Moore WR 3000 0.625
Austin Seferian-Jenkins TE 3600 17.95
Rob Gronkowski TE 7500 25.6
Broncos  DST 3600 16.0
50000
180.505


### What happened??
As we can see, we did not get the same result, and in fact got a much worse result. Whereas in greedy algorithms we only had to worry about the prioritization, there are many more parameters involved with a genetic algorithm. We can change how many generations we evolve, the size of each population, the fitness function, how teams are bred, how teams are mutated, what percentage of teams are bred and what percentage are thrown away at the end of each generation, and more. Altering these values is called "tuning the parameters," and finding the best values for these parameters is non-trivial.

### Feedback
I welcome any and all feeback on the code, explanations, layout, etc. I can be contacted at [sambrady3@gmail.com](mailto:sambrady3@gmail.com)